In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filter users and books
user_rating_counts = ratings['user_id'].value_counts()
ratings = ratings[ratings['user_id'].isin(user_rating_counts[user_rating_counts >= 200].index)]

book_rating_counts = ratings['book_id'].value_counts()
ratings = ratings[ratings['book_id'].isin(book_rating_counts[book_rating_counts >= 100].index)]

# Merge with book titles
df = ratings.merge(books, on='book_id')

# Create pivot table
pivot = df.pivot_table(index='title', columns='user_id', values='rating').fillna(0)

print("Pivot table shape:", pivot.shape)

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(pivot.values)

print("Model trained successfully!")

In [ ]:
def get_recommends(book_title):
    if book_title not in pivot.index:
        return f"Book '{book_title}' not found in dataset."

    book_idx = list(pivot.index).index(book_title)
    distances, indices = model.kneighbors(
        pivot.iloc[book_idx, :].values.reshape(1, -1),
        n_neighbors=6
    )

    recommendations = []
    for i in range(1, len(distances[0])):
        title = pivot.index[indices[0][i]]
        distance = distances[0][i]
        recommendations.append([title, distance])

    recommendations.reverse()
    return [book_title, recommendations]

In [ ]:
# Test with a sample book
test_result = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(test_result)